In [1]:
# Copyright 2021 Supun Nakandala. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

import os
import sys
import numpy as np

import tensorflow
if int(tensorflow.__version__.split(".")[0]) >= 2:
    import tensorflow.compat.v1 as tf
else:
    import tensorflow as tf


import pandas as pd
import random
import math
import argparse

sys.path.append('./')
import os
import h5py
import pathlib
import logging
import numpy as np
import pandas as pd

import tensorflow
if int(tensorflow.__version__.split(".")[0]) >= 2:
    import tensorflow.compat.v1 as tf
else:
    import tensorflow as tf

from datetime import datetime
import multiprocessing
import argparse
import json
import pathlib
# from tqdm import tqdm
import sys
# sys.path.append(pathlib.Path(__file__).parent.absolute())
from commons import input_iterator

logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

os.chdir('/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021/')

from commons import cnn_bi_lstm_model, input_iterator, CNNBiLSTMModel, CNNModel

# Setting random seeds
tf.random.set_random_seed(2019)
random.seed(2019)
np.random.seed(2019)
import shutil

/home/animeshkumar/anaconda3/envs/deep_postures/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_train_ops(y, logits, learning_rate, n_classes, class_weights):
    y = tf.reshape(y, [-1])
    logits = tf.reshape(logits, [-1, n_classes])
    balanced_accuracy, update_op = tf.metrics.mean_per_class_accuracy(y, tf.argmax(logits, 1), n_classes)
    y = tf.reshape(tf.one_hot(y, depth=n_classes, axis=1), [-1, n_classes])

    loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y) * tf.reduce_sum(tf.constant(class_weights, dtype=tf.float32) * y, axis=1))

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss)

    return train_op, update_op, balanced_accuracy, loss


def window_generator(data_root, win_size_10s, subject_ids):
    x_segments = []; y_segments = []
    for subject_id in subject_ids:
        for x_seq, _, y_seq in input_iterator(data_root, subject_id, train=True):
            x_window = []; y_window = []
            for x,y in zip(x_seq, y_seq):
                x_window.append(x)
                y_window.append(y)

                if len(y_window) == win_size_10s:
                    yield np.stack(x_window, axis=0), np.stack(y_window, axis=0)
                    x_window = []; y_window = []


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim

In [4]:
class IterDataset(torch.utils.data.IterableDataset):
    def __init__(self, generator):
        self.generator = generator

    def __iter__(self):
        return self.generator

In [5]:
import torch.nn.functional as F
class Conv2dSame(torch.nn.Conv2d):

    def calc_same_pad(self, i: int, k: int, s: int, d: int) -> int:
        return max((math.ceil(i / s) - 1) * s + (k - 1) * d + 1 - i, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        ih, iw = x.size()[-2:]

        pad_h = self.calc_same_pad(i=ih, k=self.kernel_size[0], s=self.stride[0], d=self.dilation[0])
        pad_w = self.calc_same_pad(i=iw, k=self.kernel_size[1], s=self.stride[1], d=self.dilation[1])

        if pad_h > 0 or pad_w > 0:
            x = F.pad(
                x, [pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2]
            )
        return F.conv2d(
            x,
            self.weight,
            self.bias,
            self.stride,
            self.padding,
            self.dilation,
            self.groups,
        )

In [6]:
# x_np = np.random.rand(1, 1, 1, 1)
# x = tf.constant(x_np)

In [7]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Argument parser for training CNN model.')
    optional_arguments = parser._action_groups.pop()
    required_arguments = parser.add_argument_group('required arguments')
    required_arguments.add_argument('--pre-processed-dir', help='Pre-processed data directory', required=False, default="/home/animeshkumar/workspace/adalab/workspace/DeepPostures/MSSE-2021_pytorch/preprocessed_new")

    optional_arguments.add_argument('--transfer-learning-model', help='Transfer learning model name (default: CHAP_ALL_ADULTS)', default=None, required=False, choices=['CHAP_ALL_ADULTS', 'CHAP_AUSDIAB', 'NONE'])
    optional_arguments.add_argument('--learning-rate', help='Learning rate for training the model (default: 0.0001)', default=1e-4, type=float, required=False)
    optional_arguments.add_argument('--num-epochs', help='Number of epochs to train the model (default: 15)', default=15, type=int, required=False)
    optional_arguments.add_argument('--batch-size', help='Training batch size (default: 16)', default=16, type=int, required=False)
    
    optional_arguments.add_argument('--amp-factor', help='Factor to increase the number of neurons in the CNN layers (default: 2)', default=2, type=int, required=False)
    optional_arguments.add_argument('--cnn-window-size', help='CNN window size in seconds on which the predictions to be made (default: 10)', default=10, type=int, required=False)
    optional_arguments.add_argument('--bi-lstm-window-size', help='BiLSTM window size in minutes on which the predictions to be smoothed (default: 7)', default=7, type=int, required=False)
    
    optional_arguments.add_argument('--shuffle-buffer-size', help='Training data shuffle buffer size in terms of number of records (default: 10000)', default=10000, type=int, required=False)
    optional_arguments.add_argument('--training-data-fraction', help='Percentage of subjects to be used for training (default: 60)', default=60, type=int, required=False)
    optional_arguments.add_argument('--validation-data-fraction', help='Percentage of subjects to be used for validation (default: 20)', default=20, type=int, required=False)
    optional_arguments.add_argument('--testing-data-fraction', help='Percentage of subjects to be used for testing (default: 20)', default=20, type=int, required=False)
    optional_arguments.add_argument('--model-checkpoint-path', help='Path where the trained model will be saved (default: ./model-checkpoint)', default='./model-checkpoint', required=False)
    
    optional_arguments.add_argument('--num-classes', help='Number of classes in the training dataset (default: 2)', default=2, type=int, required=False)
    optional_arguments.add_argument('--class-weights', help='Class weights for loss aggregation (default: [1.0, 1.0])', default='[1.0, 1.0]', required=False)
    optional_arguments.add_argument('--down-sample-frequency', help='Downsample frequency in Hz for GT3X data (default: 10)', default=10, type=int, required=False)
    optional_arguments.add_argument('--silent', help='Whether to hide info messages', default=False, required=False, action='store_true')
    parser._action_groups.append(optional_arguments)
    args = parser.parse_known_args()
    args = args[0]
    # print(args)

    if os.path.exists(args.model_checkpoint_path):
        shutil.rmtree(args.model_checkpoint_path)
        # raise Exception('Model checkpoint: {} already exists.'.format(args.model_checkpoint_path))

    if args.transfer_learning_model:
        if args.transfer_learning_model == 'CHAP_ALL_ADULTS':
            args.amp_factor = 2
            args.cnn_window_size = 10
            args.bi_lstm_window_size = 7
        elif args.transfer_learning_model == 'CHAP_AUSDIAB':
            args.amp_factor = 4
            args.cnn_window_size = 10
            args.bi_lstm_window_size = 9
        elif args.transfer_learning_model != 'NONE':
            raise Exception('Unsupported transfer learning model: {}'.format(args.transfer_learning_model))
    
    assert (args.training_data_fraction + args.validation_data_fraction + args.testing_data_fraction) == 100, 'Train, validation,test split fractions should add up to 100%'
    
    subject_ids = [fname.split('.')[0] for fname in os.listdir(args.pre_processed_dir)]
    random.shuffle(subject_ids)

    n_train_subjects = int(math.ceil(len(subject_ids) * args.training_data_fraction / 100.))
    train_subjects = subject_ids[:n_train_subjects]
    subject_ids = subject_ids[n_train_subjects:]

    if (100.0 - args.training_data_fraction) > 0:
        test_frac = args.testing_data_fraction / (100.0 - args.training_data_fraction) * 100
    else:
        test_frac = 0.0
    n_test_subjects = int(math.ceil(len(subject_ids) * test_frac / 100.))
    test_subjects = subject_ids[:n_test_subjects]
    valid_subjects = subject_ids[n_test_subjects:]  

    output_shapes = ((args.bi_lstm_window_size*(60//args.cnn_window_size), args.cnn_window_size*args.down_sample_frequency, 3), (args.bi_lstm_window_size*(60//args.cnn_window_size)))
    bi_lstm_win_size = 60//args.down_sample_frequency * args.bi_lstm_window_size
    train_dataset = tf.data.Dataset.from_generator(lambda: window_generator(args.pre_processed_dir, bi_lstm_win_size, train_subjects),output_types=(tf.float32, tf.int32),
                output_shapes=output_shapes).shuffle(args.shuffle_buffer_size).batch(args.batch_size).prefetch(10)
    if valid_subjects:
        valid_dataset = tf.data.Dataset.from_generator(lambda: window_generator(args.pre_processed_dir, bi_lstm_win_size, valid_subjects),output_types=(tf.float32, tf.int32),
                output_shapes=output_shapes).batch(args.batch_size).prefetch(10)
    else:
        valid_dataset = None

    if test_subjects:
        test_dataset = tf.data.Dataset.from_generator(lambda: window_generator(args.pre_processed_dir, bi_lstm_win_size, test_subjects),output_types=(tf.float32, tf.int32),
                output_shapes=output_shapes).batch(args.batch_size).prefetch(10)
    else:
        test_dataset = None
    
    iterator =  tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)

    train_init_op = iterator.make_initializer(train_dataset)

    x, y = iterator.get_next()
    
    x = tf.reshape(x, [-1, args.cnn_window_size*args.down_sample_frequency, 3, 1])
    x = tf.identity(x, name='input')
    y = tf.reshape(y, [-1, bi_lstm_win_size])

    class_weights = eval(args.class_weights)   
    amp_factor = 2
    bi_lstm_window_size  = args.bi_lstm_window_size
    conv1 = tf.layers.conv2d(x, filters=32*amp_factor, kernel_size=[3, 3],
                                 data_format='channels_last',
                                 padding= "same", use_bias=False)

    def cnn_model_2(x, amp_factor=2):
        conv1 = tf.layers.conv2d(x, filters=32*amp_factor, kernel_size=[5, 3],
                                data_format='channels_last', padding= "same",
                                strides=(2, 1),
                                activation=tf.nn.relu)
        pool1 = conv1

        conv2 = tf.layers.conv2d(pool1, filters=64*amp_factor, kernel_size=[5, 1],
                                data_format='channels_last', padding= "same",
                                strides=(2, 1),
                                activation=tf.nn.relu)
        pool2 = conv2
        return pool2
    
    def cnn_model_3(x, amp_factor=2):
        conv1 = tf.layers.conv2d(x, filters=32*amp_factor, kernel_size=[5, 3],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 activation=tf.nn.relu)
        pool1 = conv1

        conv2 = tf.layers.conv2d(pool1, filters=64*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 activation=tf.nn.relu)
        pool2 = conv2

        conv3 = tf.layers.conv2d(pool2, filters=128*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 activation=tf.nn.relu)
        pool3 = conv3
        return pool3
    
    def cnn_model_4(x, amp_factor=2):
        conv1 = tf.layers.conv2d(x, filters=32*amp_factor, kernel_size=[5, 3],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 activation=tf.nn.relu)
        pool1 = conv1

        conv2 = tf.layers.conv2d(pool1, filters=64*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 activation=tf.nn.relu)
        pool2 = conv2

        conv3 = tf.layers.conv2d(pool2, filters=128*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 activation=tf.nn.relu)
        pool3 = conv3

        conv4 = tf.layers.conv2d(pool3, filters=256*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                strides=(2, 1), 
                                activation=tf.nn.relu)
        pool4 = conv4
        return pool4
    
    def cnn_model_5(x, amp_factor=2):
        conv1 = tf.layers.conv2d(x, filters=32*amp_factor, kernel_size=[5, 3],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 activation=tf.nn.relu)
        pool1 = conv1

        conv2 = tf.layers.conv2d(pool1, filters=64*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 activation=tf.nn.relu)
        pool2 = conv2

        conv3 = tf.layers.conv2d(pool2, filters=128*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 activation=tf.nn.relu)
        pool3 = conv3

        conv4 = tf.layers.conv2d(pool3, filters=256*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                strides=(2, 1), 
                                activation=tf.nn.relu)
        pool4 = conv4

        conv5 = tf.layers.conv2d(pool4, filters=256*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                strides=(2, 1), 
                                activation=tf.nn.relu)
        pool5 = conv5
        return pool5  
    
    def cnn_model_fc(x, amp_factor=2):
        conv1 = tf.layers.conv2d(x, filters=32*amp_factor, kernel_size=[5, 3],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 activation=tf.nn.relu)
        pool1 = conv1

        conv2 = tf.layers.conv2d(pool1, filters=64*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 activation=tf.nn.relu)
        pool2 = conv2

        conv3 = tf.layers.conv2d(pool2, filters=128*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 activation=tf.nn.relu)
        pool3 = conv3

        conv4 = tf.layers.conv2d(pool3, filters=256*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                strides=(2, 1), 
                                activation=tf.nn.relu)
        pool4 = conv4

        conv5 = tf.layers.conv2d(pool4, filters=256*amp_factor, kernel_size=[5, 1],
                                 data_format='channels_last', padding= "same",
                                strides=(2, 1), 
                                activation=tf.nn.relu)
        pool5 = conv5        
        pool5 = tf.transpose(pool5, [0, 3, 1, 2])
        size = pool5.shape[-1] * pool5.shape[-2] * pool5.shape[-3]

        logits = tf.layers.dense(tf.reshape(pool5,(-1, size)), units=256*amp_factor)
        return logits
    
    def cnn_bi_lstm_model(x, amp_factor, bil_lstm_win_size, num_classes):
        logits = cnn_model_fc(x, amp_factor=amp_factor)
        logits = tf.reshape(logits, [-1, bil_lstm_win_size, 256*amp_factor])

        forward_cell = tf.nn.rnn_cell.LSTMCell(128)
        backward_cell = tf.nn.rnn_cell.LSTMCell(128)
        encoder_outputs,_ = tf.nn.bidirectional_dynamic_rnn(
                forward_cell,
                backward_cell,
                logits,
                dtype=tf.float32
            )
        encoder_outputs = tf.concat(encoder_outputs, axis=2)
        logits = tf.layers.dense(encoder_outputs, units=num_classes)
        logits = tf.reshape(logits, [-1, bil_lstm_win_size, num_classes])
        return logits

    cnn2 = cnn_model_2(x)
    cnn3 = cnn_model_3(x)
    cnn4 = cnn_model_4(x)
    cnn5 = cnn_model_5(x)
    cnnfc = cnn_model_fc(x)
    num_classes = 2
    cnn_bi_lstm = cnn_bi_lstm_model(x, amp_factor, bi_lstm_window_size, num_classes)


   
 
    conv1_pt = Conv2dSame(in_channels=1, out_channels=32*amp_factor, kernel_size=(5, 3), stride=(2, 1))
    conv2_pt = Conv2dSame(in_channels=32*amp_factor, out_channels=64*amp_factor, kernel_size=(5, 1), stride=(2, 1))
    conv3_pt = Conv2dSame(in_channels=64*amp_factor, out_channels=128*amp_factor, kernel_size=(5, 1), stride=(2, 1))
    conv4_pt = Conv2dSame(in_channels=128*amp_factor, out_channels=256*amp_factor, kernel_size=(5, 1), stride=(2, 1))
    conv5_pt = Conv2dSame(in_channels=256*amp_factor, out_channels=256*amp_factor, kernel_size=(5, 1), stride=(2, 1))
    # size = 
    fc = nn.Linear(256*amp_factor * 3 * 16, 256*amp_factor)
    py_cnn = CNNModel(amp_factor=2)
    lstm = nn.LSTM(input_size=256*amp_factor, hidden_size=128, bidirectional=True, batch_first=True)
    fc_lstm = nn.Linear(128*amp_factor, 2)
    
        
    with tf.Session() as sess:
        sess.run(train_init_op)
        sess.run(tf.global_variables_initializer())

        print("Layer 1")
        print("Input", np.array(x.eval()).shape)
        output = sess.run(conv1)
        output = np.array(output)
        print("Output", output.shape)
        output_transposed = np.transpose(output, (0, 3, 1, 2))
        print("Output transposed", output_transposed.shape)

        x_list = x.eval()
        x_np= np.transpose(x_list, (0, 3, 1, 2)).copy()
        x_pt = torch.tensor(x_np, dtype=torch.float32)
        output_pt = conv1_pt(x_pt)
        print("Output pt", output_pt.detach().numpy().shape)

        print("Layer 2")
        print("Input", output.shape)
        output2 = sess.run(cnn2)
        output2 = np.array(output2)
        print("Output", output2.shape)
        print("Output transposed", np.transpose(output2, (0, 3, 1, 2)).shape)

        output_pt2 = conv2_pt(output_pt)
        print("Output pt", output_pt2.detach().numpy().shape)

        print("Layer 3")
        print("Input", output2.shape)
        output3 = sess.run(cnn3)
        output3 = np.array(output3)
        print("Output", output3.shape)
        print("Output transposed", np.transpose(output3, (0, 3, 1, 2)).shape)

        output_pt3 = conv3_pt(output_pt2)
        print("Output pt", output_pt3.detach().numpy().shape)

        print("Layer 4")
        print("Input", output3.shape)
        output4 = sess.run(cnn4)
        output4 = np.array(output4)
        print("Output", output4.shape)
        print("Output transposed", np.transpose(output4, (0, 3, 1, 2)).shape)

        output_pt4 = conv4_pt(output_pt3)
        print("Output pt", output_pt4.detach().numpy().shape)

        print("Layer 5")
        print("Input", output4.shape)
        output5 = sess.run(cnn5)
        output5 = np.array(output5)
        print("Output", output5.shape)
        print("Output transposed", np.transpose(output5, (0, 3, 1, 2)).shape)

        output_pt5 = conv5_pt(output_pt4)
        print("Output pt", output_pt5.detach().numpy().shape)

        k_pt = output_pt5.view(output_pt5.size(0), -1)
        print("pytorch", k_pt.size())

        pool5 = tf.transpose(output5, [0, 3, 1, 2])
        size = pool5.shape[-1] * pool5.shape[-2] * pool5.shape[-3]
        k = tf.reshape(pool5,(-1, size))
        print(k.shape)

        print("Layer 6")
        print("Input", output5.shape)
        outputcnn = sess.run(cnnfc)
        outputcnn = np.array(outputcnn)
        print("Output", outputcnn.shape)
        # print("Output transposed", np.transpose(output5, (0, 3, 1, 2)).shape)

        output_cnn_pt = py_cnn(x_pt)
        print("Output pt", output_cnn_pt.detach().numpy().shape)

        print("Layer LSTM")
        print("Input", outputcnn.shape)
        outputlstm = sess.run(cnn_bi_lstm)
        print("Output", outputlstm.shape)

        output_cnn_pt = output_cnn_pt.view(-1, bi_lstm_window_size, 256*amp_factor)
        # print(output_cnn_pt.size())
        outputlstm_pt = lstm(output_cnn_pt)
        print("Output pt ", outputlstm_pt[0].size())
        output_lstm_fc = fc_lstm(outputlstm_pt[0])
        output_lstm_fc = output_lstm_fc.view(-1, bi_lstm_window_size, 2)
        print("Output pt fc", output_lstm_fc.size(), output_lstm_fc)

        print("Loss Calculation")
        print("y shape value", np.array(y.eval()))
        print("y shape", np.array(y.eval()).shape)
        print("logits shape", outputlstm.shape)
        y = tf.reshape(y, [-1])
        logits = tf.reshape(outputlstm, [-1, num_classes])

        print("y reshape value", np.array(y.eval()))
        print("y reshape ", np.array(y.eval()).shape)
        print("logits reshape", logits.shape)

        balanced_accuracy, update_op = tf.metrics.mean_per_class_accuracy(y, tf.argmax(logits, 1), num_classes)
        y = tf.reshape(tf.one_hot(y, depth=num_classes, axis=1), [-1, num_classes])
        print("y one hot", np.array(y.eval()))
        print("y reshape ", np.array(y.eval()).shape)

        loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y) * tf.reduce_sum(tf.constant(class_weights, dtype=tf.float32) * y, axis=1))
        print(loss)

    








# # N H W C


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.


Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.


Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


...

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



2024-05-08 13:33:50.295325: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2024-05-08 13:33:52.210148: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-05-08 13:33:52.210221: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (animeshkumar-msi): /proc/driver/nvidia/version does not exist


2024-05-08 13:33:52.211529: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2024-05-08 13:33:52.245051: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2799925000 Hz
2024-05-08 13:33:52.246169: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5617b94b48f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-05-08 13:33:52.246220: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version



Layer 1
Input (672, 100, 3, 1)
Output (672, 100, 3, 64)
Output transposed (672, 64, 100, 3)
Output pt (672, 64, 50, 3)
Layer 2
Input (672, 100, 3, 64)
Output (672, 25, 3, 128)
Output transposed (672, 128, 25, 3)
Output pt (672, 128, 25, 3)
Layer 3
Input (672, 25, 3, 128)
Output (672, 13, 3, 256)
Output transposed (672, 256, 13, 3)
Output pt (672, 256, 13, 3)
Layer 4
Input (672, 13, 3, 256)
Output (672, 7, 3, 512)
Output transposed (672, 512, 7, 3)
Output pt (672, 512, 7, 3)
Layer 5
Input (672, 7, 3, 512)
Output (672, 4, 3, 512)
Output transposed (672, 512, 4, 3)
Output pt (672, 512, 4, 3)
pytorch torch.Size([672, 6144])
(672, 6144)
Layer 6
Input (672, 4, 3, 512)
Output (672, 512)
Output pt (672, 512)
Layer LSTM
Input (672, 512)
Output (96, 7, 2)
Output pt  torch.Size([96, 7, 256])
Output pt fc torch.Size([96, 7, 2]) tensor([[[ 0.0708, -0.0522],
         [ 0.0714, -0.0490],
         [ 0.0712, -0.0457],
         ...,
         [ 0.0700, -0.0406],
         [ 0.0685, -0.0382],
         [ 0.

y one hot [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
y reshape  (672, 2)
Tensor("Sum_1:0", shape=(), dtype=float32)
